In [17]:
import pickle
import pandas as pd
import numpy as np
from pre_processing import preprocess
from sklearn.impute import SimpleImputer as si

In [35]:
test_df = pd.read_csv("/Users/prajualpillai/Desktop/prajual/Personal_git/boosting/test.csv")
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [36]:
imp_median = si(missing_values=np.nan, strategy="median")
imp_mode = si(missing_values=np.nan, strategy="most_frequent")
test_df["Cabin"] = test_df["Cabin"].astype(str)
test_df["PassengerId"] = test_df["PassengerId"].astype(str)
for col in test_df.columns:
    print(col)
    if type(test_df[col][0]) == str:
        imp_mode.fit(test_df[[col]])
        test_df[[col]] = imp_mode.transform(test_df[[col]])
    else:
        imp_median.fit(test_df[[col]])
        test_df[[col]] = imp_median.transform(test_df[[col]])

PassengerId
Pclass
Name
Sex
Age
SibSp
Parch
Ticket
Fare
Cabin
Embarked


In [37]:
pre_obj = preprocess()
processed_df = pre_obj.main(test_df)

In [38]:
processed_df

,PassengerId,Pclass,Age,SibSp,Parch,Ticket,Fare,em_1,em_2,em_3,Sex_1,Sex_2
0,892,3.0,34.5,0.0,0.0,152,7.8292,0.0,1.0,0.0,0.0,1.0
1,893,3.0,47.0,1.0,0.0,221,7.0000,0.0,0.0,1.0,1.0,0.0
2,894,2.0,62.0,0.0,0.0,73,9.6875,0.0,1.0,0.0,0.0,1.0
3,895,3.0,27.0,0.0,0.0,147,8.6625,0.0,0.0,1.0,0.0,1.0
4,896,3.0,22.0,1.0,1.0,138,12.2875,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3.0,27.0,0.0,0.0,267,8.0500,0.0,0.0,1.0,0.0,1.0
414,1306,1.0,39.0,0.0,0.0,324,108.9000,1.0,0.0,0.0,1.0,0.0
415,1307,3.0,38.5,0.0,0.0,346,7.2500,0.0,0.0,1.0,0.0,1.0
416,1308,3.0,27.0,0.0,0.0,220,8.0500,0.0,0.0,1.0,0.0,1.0


In [39]:
test_df.isnull().sum()

PassengerId    0
Pclass         0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
em_1           0
em_2           0
em_3           0
Sex_1          0
Sex_2          0
dtype: int64

In [40]:
len(test_df)

418

In [41]:
with open('clf.pickle', 'rb') as f:
    clf = pickle.load(f)

In [42]:
y = clf.predict(processed_df)

In [43]:
result = processed_df[["PassengerId"]]
result["Survived"] = y

/var/folders/05/mz9vyrj559z6gnm1ptm7x5z80000gq/T/ipykernel_64042/1281067289.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["Survived"] = y


In [44]:
result

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [45]:
result.to_csv("titanic_result.csv", index=False)